# Atlas da Violência 2024

O Atlas da Violência produzido pelo Instituto de Pesquisa Econômica Aplicada (IPEA), em parceria com o Fórum Brasileiro de Segurança Pública (FBSP), é divulgado anualmente com dois anos de defasagem. <br>
As [publicações](https://www.ipea.gov.br/atlasviolencia/publicacoes) e [estatísticas](https://www.ipea.gov.br/atlasviolencia/filtros-series) estão disponíveis no portal do [IPEA](portal).

Com um ligeiro intervalo, as séries históricas vão de 1989 a 1995 e de 2000 a 2022.

In [7]:
awk -F";" 'NR>1 {print $3}' taxa_homicidios_2024.csv | sort -u | pr -6



2024-06-19 11:20                                                Página 1


1989	    1994	2003	    2008	2013	    2018
1990	    1995	2004	    2009	2014	    2019
1991	    2000	2005	    2010	2015	    2020
1992	    2001	2006	    2011	2016	    2021
1993	    2002	2007	    2012	2017	    2022


























































## Inspeção dos dados

### Visualizamos a série histórica da taxa de homicídios:

In [16]:
 awk 'NR <= 10' taxa_homicidios_2024.csv

cod;nome;período;valor
5213756;Montividiu;1989;0
5213707;"Montes Claros de Goiás";1989;11.63
5213509;"Monte Alegre de Goiás";1989;12.73
5213400;Moiporá;1989;0
5213103;Mineiros;1989;0
5213087;Minaçu;1989;3.18
5213053;"Mimoso de Goiás";1989;0
5213004;Maurilândia;1989;0
5212956;Matrinchã;1989;0


### Ordenamos em ordem descendente:

In [20]:
 awk -F";" 'BEGIN {OFS=";"} ($3 == 2022) {print $0}' taxa_homicidios_2024.csv | \
 sort -t";" -k4,4gr | \
 awk 'NR <= 10' | \
 awk -F";" 'BEGIN {print "cod;nome;período;valor"} { print $0}' 

cod;nome;período;valor
1501402;Belém;2022;7179.16
2109908;"Santa Inês";2022;2402.75
1400100;"Boa Vista";2022;2320.84
5002704;"Campo Grande";2022;2087.68
1200401;"Rio Branco";2022;2050.72
2305506;Iguatu;2022;1585.82
3171303;Viçosa;2022;1372.12
3547809;"Santo André";2022;1334.86
2903409;Belmonte;2022;1015.8
4110706;Irati;2022;821.65


### Descrição da tabela:

A tabela contém quatro colunas:<br>
a primeira informa o código do município, de acordo com o padrão do IBGE;<br>
a segunda, o nome do município;<br>
a terceira, o período - que corresponde ao ano da apuração;<br>
a quarta e última, o valor - que corresponde à taxa de homicídios apurada.<br>

### Cálculo da taxa de homicídios

A taxa de homicídios é um indicador calculado de acordo com a fórmula:<br>
<center>$Óbitos/População$ $ * 100.000$<br><br>

Utilizou-se a tabela de população divulgada pelo [IBGE](https://www.ibge.gov.br/estatisticas/sociais/populacao/22827-censo-demografico-2022.html?edicao=37225&t=resultados) correspondente ao censo demográfico realizado em 2022.


### Visualizamos a tabela do censo demográfico 2022

In [22]:
 awk 'NR <= 10' Censo_PNAD2022_20231222.csv

;UF;COD. UF;COD. MUNIC;NOME DO MUNICÍPIO;POP. COLETADA;POP. IMPUTADA;POP. TOTAL
;RO;11;00015;Alta Floresta D'Oeste; 20 865;  629; 21 494
;RO;11;00023;Ariquemes; 93 006; 3 827; 96 833
;RO;11;00031;Cabixi; 5 100;  251; 5 351
;RO;11;00049;Cacoal; 84 923; 1 964; 86 887
;RO;11;00056;Cerejeiras; 15 297;  593; 15 890
;RO;11;00064;Colorado do Oeste; 14 962;  701; 15 663
;RO;11;00072;Corumbiara; 7 164;  355; 7 519
;RO;11;00080;Costa Marques; 12 511;  116; 12 627
;RO;11;00098;Espigão D'Oeste; 28 182; 1 232; 29 414


Removemos a primeira, a sexta e a sétima colunas e substituímos o separador de milhar:

In [4]:
awk -F';' '{OFS=";"} NR==1 { print $2,$3,$4,$5,$8 } 
NR>1 { gsub(/ /, "", $8); print $2,$3,$4,$5,$8 }' Censo_PNAD2022_20231222.csv | \
sort -t";" -k5,5n | head


UF;COD. UF;COD. MUNIC;NOME DO MUNICÍPIO;POP. TOTAL
MG;31;66600;Serra da Saudade;833
SP;35;07209;Borá;907
GO;52;01207;Anhanguera;924
MT;51;01209;Araguainha;1010
SP;35;32868;Nova Castilho;1062
MG;31;15607;Cedro do Abaeté;1081
RS;43;00661;André da Rocha;1135
TO;17;15507;Oliveira de Fátima;1164
RS;43;22350;União da Serra;1170


### Junção das tabelas

Na tabela "taxa_homicidios_2024.csv", os dois primeiros dígitos do código do município contém o Código da UF, de acordo com a tabela do IBGE. Na tabela do Censo 2022, os códigos da UF e do Município são apresentados em colunas separadas.<br>

Primeiro, salvamos as modificações anteriores em um arquivo temporário:

In [7]:
awk -F';' '{OFS=";"} NR==1 { print $2,$3,$4,$5,$8 } 
NR>1 { gsub(/ /, "", $8); print $2,$3,$4,$5,$8 }' Censo_PNAD2022_20231222.csv | \
sort -t";" -k5,5n > temp.csv

Depois, unificamos os códigos da UF e Município em uma nova coluna, para padronizar a disposição das duas tabelas:

In [10]:
awk -F';' 'BEGIN{OFS=";"} 
NR==1 {print $1, "cod", "nome", "pop_total"} 
NR>1 {print $1, $2 $3, $4, $5}' temp.csv | head

UF;cod;nome;pop_total
MG;3166600;Serra da Saudade;833
SP;3507209;Borá;907
GO;5201207;Anhanguera;924
MT;5101209;Araguainha;1010
SP;3532868;Nova Castilho;1062
MG;3115607;Cedro do Abaeté;1081
RS;4300661;André da Rocha;1135
TO;1715507;Oliveira de Fátima;1164
RS;4322350;União da Serra;1170


Salvamos a saída em um novo arquivo e removemos o arquivo temporário:

In [13]:
awk -F';' 'BEGIN{OFS=";"} 
NR==1 {print $1, "cod", "nome", "pop_total"} 
NR>1 {print $1, $2 $3, $4, $5}' temp.csv > censo_2022.csv
rm -f temp.csv

Agora, adicionamos a coluna "pop_total" da tabela do Censo 2022 ao arquivo que contém a tabela da taxa de homicídios:

In [25]:
join -1 2 -2 1 -t";" <(sort -t";" -k2 censo_2022.csv) \
<(sort -t";" -k1 taxa_homicidios_2024.csv) | \
awk '{FS=OFS=";"} {print $1, $2, $3, $4, $6, $7}' | \
sort -t';' -k6,6gr | \
awk 'BEGIN {F=OFS=";"} NR == 1 {print "Cod_Mun;UF;Município;População;Período;Taxa"} 
NR > 1 { print $0}' | head

Cod_Mun;UF;Município;População;Período;Taxa
2407906;RN;Monte das Gameleiras;2276;2020;8925
1501402;PA;Belém;1303403;2022;7179.16
3550506;SP;São Pedro do Turvo;7217;2020;4937
1600253;AP;Itaubal;5599;2020;4717
4209177;SC;Jupiá;2555;2020;3298
2611705;PE;Riacho das Almas;20639;2020;3003
4113759;PR;Lunardelli;4872;2020;2592
3513850;SP;Dirce Reis;1620;2020;2445
2109908;MA;Santa Inês;85014;2022;2402.75


Salvamos a saída em um novo arquivo:

In [26]:
join -1 2 -2 1 -t";" <(sort -t";" -k2 censo_2022.csv) \
<(sort -t";" -k1 taxa_homicidios_2024.csv) | \
awk '{FS=OFS=";"} {print $1, $2, $3, $4, $6, $7}' | \
sort -t';' -k6,6gr | \
awk 'BEGIN {F=OFS=";"} NR == 1 {print "Cod_Mun;UF;Município;População;Período;Taxa"} 
NR > 1 { print $0}' > th2024.csv

De acordo com notícia veiculada pelo G1 Bahia, Santo Antônio de Jesus, município situado no recôncavo baiano, possui a maior taxa de homicídios do Brasil.

<center>
<img src="oglobo_1.png" width="360" height="180">
</center>



A taxa de homicídios e a população do município mencionadas na matéria condizem com as informações do Atlas e do Censo:
<center>
<img src="oglobo_2.png" width="360" height="180">
</center>


In [35]:
awk -F";" '($3 ~ /Santo Antônio de Jesus/ && $5 == 2022) {print $0}' th2024.csv | head

2928703;BA;Santo Antônio de Jesus;103055;2022;93.15


Contudo, quando ordenamos os municípios do Estado da Bahia em ordem descendente, pela taxa de homicídios, para o ano de 2022, os dados apresentam anomalias:

In [36]:
awk -F";" '($2 ~ /BA/ && $5 == 2022) {print $0}' th2024.csv | head

2903409;BA;Belmonte;20121;2022;1015.8
2910503;BA;Entre Rios;38098;2022;587.89
2925006;BA;Planalto;23334;2022;364.55
2906501;BA;Candeias;72382;2022;357.12
2933208;BA;Vera Cruz;42529;2022;202.17
2902401;BA;Aurelino Leal;11179;2022;178.91
2928950;BA;São Domingos;8426;2022;154.14
2931103;BA;Tanquinho;7717;2022;142.54
2915007;BA;Itaeté;13472;2022;126.19
2932309;BA;Ubatã;16094;2022;124.27


O mesmo acontece quando classificamos em nível nacional, como já visto anteriormente:

In [37]:
awk -F";" '{print $0}' th2024.csv | head

Cod_Mun;UF;Município;População;Período;Taxa
2407906;RN;Monte das Gameleiras;2276;2020;8925
1501402;PA;Belém;1303403;2022;7179.16
3550506;SP;São Pedro do Turvo;7217;2020;4937
1600253;AP;Itaubal;5599;2020;4717
4209177;SC;Jupiá;2555;2020;3298
2611705;PE;Riacho das Almas;20639;2020;3003
4113759;PR;Lunardelli;4872;2020;2592
3513850;SP;Dirce Reis;1620;2020;2445
2109908;MA;Santa Inês;85014;2022;2402.75
